In [1]:
!pip install streamlit googletrans==4.0.0-rc1
!pip install googletrans==3.1.0a0
!pip install nltk
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
from timeit import default_timer as timer
import streamlit as st
import googletrans
from googletrans import Translator
from google.colab import drive
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 490.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.4 MB/s eta 0:00:00
  Created wheel for 

In [2]:
translator = Translator()

# Define dataset path
dataset_path = "cfilt/iitb-english-hindi"

# Load dataset from 800,000 to 1,200,000 lines
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [3]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [4]:
translatedEngHin = []
for index, (eng, hin) in enumerate(zip(english_sentences, hindi_sentences)):
    print("Index:", index)
    print("English:", eng)
    translated_eng = translator.translate(eng, src='en', dest='hi').text
    print("Translated: ", translated_eng)
    print("Original: ", hin)
    print("------------------------------")
    translatedEngHin.append(translated_eng)

Index: 0
English: Give your application an accessibility workout
Translated:  अपने एप्लिकेशन को एक्सेसिबिलिटी वर्कआउट दें
Original:  अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
------------------------------
Index: 1
English: Accerciser Accessibility Explorer
Translated:  एक्सर्साइज़र एक्सेसिबिलिटी एक्सप्लोरर
Original:  एक्सेर्साइसर पहुंचनीयता अन्वेषक
------------------------------
Index: 2
English: The default plugin layout for the bottom panel
Translated:  निचले पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट
Original:  निचले पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
Index: 3
English: The default plugin layout for the top panel
Translated:  शीर्ष पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट
Original:  ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
Index: 4
English: A list of plugins that are disabled by default
Translated:  उन प्लगइन्स की सूची जो डिफ़ॉल्ट रूप से अक्षम हैं
Original:  उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
---------------

In [5]:
# BLEU Score
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[hindi] for hindi in hindi_sentences]  # Create reference list of lists
candidates = translatedEngHin  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.4257


In [6]:
# STS Score

!pip install sentence-transformers

# STS Score using SentenceTransformer
from sentence_transformers import SentenceTransformer, util
import torch

# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(hindi_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedEngHin, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.7624


In [7]:
# CHRF Score

!pip install sacrebleu
from sacrebleu.metrics import CHRF

# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedEngHin, [hindi_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")


CHRF score: 35.6417


In [9]:
import nltk
from nltk.translate.meteor_score import meteor_score

# Download required NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
hindi_sentences_tokenized = [nltk.word_tokenize(sent) for sent in hindi_sentences]
translatedEngHin_tokenized = [nltk.word_tokenize(sent) for sent in translatedEngHin]

# Calculate METEOR score
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(hindi_sentences_tokenized, translatedEngHin_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


METEOR score: 0.2287


In [10]:
import nltk
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Download necessary NLTK data
nltk.download('punkt')

def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
hindi_model = train_model(hindi_sentences)
english_model = train_model(translatedEngHin)

# Calculate perplexities
hindi_perplexity = calculate_perplexity(hindi_model, hindi_sentences)
english_perplexity = calculate_perplexity(english_model, translatedEngHin)

print(f"Hindi Perplexity: {hindi_perplexity:.4f}")
print(f"Translated English Perplexity: {english_perplexity:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hindi Perplexity: 11.6234
Translated English Perplexity: 10.9922
